In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

In [65]:
image = pd.read_csv("C:\Aalesh and Mandar\Datasets\Cases\Image Segmentation\Image_Segmention.csv")

In [66]:
image.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [67]:
x = image.drop(["Class"], axis=1)
y = image["Class"]

In [68]:
lebals = preprocessing.LabelEncoder()
le_y = lebals.fit_transform(y)
print(lebals.classes_)

['BRICKFACE' 'CEMENT' 'FOLIAGE' 'GRASS' 'PATH' 'SKY' 'WINDOW']


In [69]:
x_train, x_test, y_train, y_test = train_test_split(x, le_y, stratify = le_y, random_state = 2022, test_size = 0.3)

In [70]:
scaler = StandardScaler()

lda = LinearDiscriminantAnalysis()
svm = SVC(kernel="linear", probability = True,  random_state = 2022)
pipe_svm = Pipeline([("STD", scaler),("SVM",svm)])
lr = LogisticRegression()
nb = GaussianNB()

In [71]:
lda.fit(x_train, y_train)
y_pred = lda.predict(x_test)
y_pred_prob = lda.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))

0.8571428571428571


In [72]:
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
y_pred_prob = nb.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))

0.6984126984126984


In [73]:
pipe_svm.fit(x_train, y_train)
y_pred = pipe_svm.predict(x_test)
y_pred_prob = pipe_svm.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))

0.9047619047619048


In [74]:
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_prob = lr.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))

0.9047619047619048


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [75]:
voting = VotingClassifier([("LDA", lda),("NB", nb),("SVM_P",pipe_svm),("LR",lr)], voting ="soft")

In [76]:
voting.fit(x_train, y_train)
y_pred = voting.predict(x_test)
y_pred_prob = voting.predict_proba(x_test)[:,1]
print(accuracy_score(y_test, y_pred))

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.873015873015873


In [77]:
voting.get_params()

{'estimators': [('LDA', LinearDiscriminantAnalysis()),
  ('NB', GaussianNB()),
  ('SVM_P',
   Pipeline(steps=[('STD', StandardScaler()),
                   ('SVM',
                    SVC(kernel='linear', probability=True, random_state=2022))])),
  ('LR', LogisticRegression())],
 'flatten_transform': True,
 'n_jobs': None,
 'verbose': False,
 'voting': 'soft',
 'weights': None,
 'LDA': LinearDiscriminantAnalysis(),
 'NB': GaussianNB(),
 'SVM_P': Pipeline(steps=[('STD', StandardScaler()),
                 ('SVM',
                  SVC(kernel='linear', probability=True, random_state=2022))]),
 'LR': LogisticRegression(),
 'LDA__covariance_estimator': None,
 'LDA__n_components': None,
 'LDA__priors': None,
 'LDA__shrinkage': None,
 'LDA__solver': 'svd',
 'LDA__store_covariance': False,
 'LDA__tol': 0.0001,
 'NB__priors': None,
 'NB__var_smoothing': 1e-09,
 'SVM_P__memory': None,
 'SVM_P__steps': [('STD', StandardScaler()),
  ('SVM', SVC(kernel='linear', probability=True, random_state=2022

In [78]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2022)
params = {"SVM_P__SVM__C"  :  np.linspace(0, 5, 5),
          "SVM_P__SVM__gamma" : np.linspace(0.001, 5, 5),
          "LR__C":  np.linspace(0,5,5),
          }
         
gcv = GridSearchCV(voting, param_grid = params, verbose = 3, scoring = "neg_log_loss", cv = kfold)
gcv.fit(x,y)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV 1/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=0.001;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=0.001;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=0.001;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=0.001;, score=nan total time=   0.0s
[CV 5/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=0.001;, score=nan total time=   0.0s
[CV 1/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=1.2507499999999998;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=1.2507499999999998;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=1.2507499999999998;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.0, SVM_P__SVM__C=0.0, SVM_P__SVM__gamma=1.2507499999999998;, score=nan total time=   0.0s
[CV

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:301: RuntimeWarning: invalid value encountered in double_scalars
  loss += 0.5 * alpha * squared_norm(w)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:352: RuntimeWarning: invalid value encountered in multiply
  grad[:, :n_features] += alpha * w
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

[CV 4/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=0.001;, score=-0.490 total time=   0.0s
[CV 5/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=0.001;, score=-0.510 total time=   0.0s
[CV 1/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=1.2507499999999998;, score=-0.647 total time=   0.0s
[CV 2/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=1.2507499999999998;, score=-0.594 total time=   0.0s
[CV 3/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=1.2507499999999998;, score=-0.486 total time=   0.0s
[CV 4/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=1.2507499999999998;, score=-0.490 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:301: RuntimeWarning: invalid value encountered in double_scalars
  loss += 0.5 * alpha * squared_norm(w)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:352: RuntimeWarning: invalid value encountered in multiply
  grad[:, :n_features] += alpha * w
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

[CV 5/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=1.2507499999999998;, score=-0.510 total time=   0.0s
[CV 1/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=2.5004999999999997;, score=-0.647 total time=   0.0s
[CV 2/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=2.5004999999999997;, score=-0.594 total time=   0.0s
[CV 3/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=2.5004999999999997;, score=-0.486 total time=   0.0s
[CV 4/5] END LR__C=0.0, SVM_P__SVM__C=1.25, SVM_P__SVM__gamma=2.5004999999999997;, score=-0.490 total time=   0.0s


...

[CV 3/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=3.75025;, score=-0.232 total time=   0.0s
[CV 4/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=3.75025;, score=-0.272 total time=   0.0s
[CV 5/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=3.75025;, score=-0.253 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 1/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=5.0;, score=-0.459 total time=   0.0s
[CV 2/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=5.0;, score=-0.365 total time=   0.0s
[CV 3/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=5.0;, score=-0.232 total time=   0.0s
[CV 4/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=5.0;, score=-0.272 total time=   0.0s
[CV 5/5] END LR__C=5.0, SVM_P__SVM__C=5.0, SVM_P__SVM__gamma=5.0;, score=-0.253 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=VotingClassifier(estimators=[('LDA',
                                                     LinearDiscriminantAnalysis()),
                                                    ('NB', GaussianNB()),
                                                    ('SVM_P',
                                                     Pipeline(steps=[('STD',
                                                                      StandardScaler()),
                                                                     ('SVM',
                                                                      SVC(kernel='linear',
                                                                          probability=True,
                                                                          random_state=2022))])),
                                                    ('LR',
                                                     Logist

In [82]:
print(gcv.best_params_)
print(gcv.best_score_)
pd_cv = pd.DataFrame(gcv.cv_results_)
pd_cv.head()

{'LR__C': 1.25, 'SVM_P__SVM__C': 2.5, 'SVM_P__SVM__gamma': 0.001}
-0.3113961580475298


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LR__C,param_SVM_P__SVM__C,param_SVM_P__SVM__gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010401,0.000490,0.0,0.0,0.0,0.0,0.001,"{'LR__C': 0.0, 'SVM_P__SVM__C': 0.0, 'SVM_P__S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,125
1,0.010400,0.000800,0.0,0.0,0.0,0.0,1.25075,"{'LR__C': 0.0, 'SVM_P__SVM__C': 0.0, 'SVM_P__S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,104
2,0.009601,0.000489,0.0,0.0,0.0,0.0,2.5005,"{'LR__C': 0.0, 'SVM_P__SVM__C': 0.0, 'SVM_P__S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,103
3,0.010401,0.000801,0.0,0.0,0.0,0.0,3.75025,"{'LR__C': 0.0, 'SVM_P__SVM__C': 0.0, 'SVM_P__S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
4,0.010401,0.001020,0.0,0.0,0.0,0.0,5.0,"{'LR__C': 0.0, 'SVM_P__SVM__C': 0.0, 'SVM_P__S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,101
